In [40]:
import sys
sys.path.insert(1, '../Src')
import numpy as np
import pandas as pd
import os
from utils.feature_engineering import *
from utils.preprocessing import *

PATH_DATASET = '../Dataset/'
#PATH_DATASET = '../Dataset/datah-m1-challange/'
PATH_RESULTS = '../Results/'

In [41]:
df_sales  = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','sales.pkl'))
df_items  = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','item.pkl'))
df_stores = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','store.pkl'))

df_submission_sample = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','submission_sample.pkl'))

In [42]:
N_submission = df_submission_sample.shape[0]
N_sales      = df_sales.shape[0]

In [43]:
df_sales = fe_dates(df_sales)
df_submission_sample = fe_dates(df_submission_sample)

In [44]:
df_sales = df_sales.merge(df_items,on=['I100'])
df_sales = df_sales.merge(df_stores,on=['S100'])
    
df_submission_sample = df_submission_sample.merge(df_items,on=['I100'])
df_submission_sample = df_submission_sample.merge(df_stores,on=['S100'])



In [45]:
def string_to_categorical(df):
    for name in df.columns:
        aux = df[name].dtype
        if str(aux) in ['str','object']:
            df[name] = df[name].astype('category')
            print(name,aux,df[name].dtype)
    return df

In [46]:
df_sales.replace([np.inf, -np.inf, np.nan],0,inplace=True)
df_submission_sample.replace([np.inf, -np.inf, np.nan],0,inplace=True)
df_sales             = string_to_categorical(df_sales).copy()
df_submission_sample = string_to_categorical(df_submission_sample).copy()


item_id object category
ID object category
item_id object category


In [47]:
df_sales.head()

,DATE,S100,I100,C100,C101,QTT,item_id,year,month,day,...,is_year_start,is_quarter_start,is_month_start,is_month_end,I101,I102,I103,S101,S102,S103
0,2017-01-08,0,0,12,76,2,0_12_76,2017,1,8,...,False,False,False,False,2,1,1,1,17,10
1,2017-01-08,0,0,12,149,3,0_12_149,2017,1,8,...,False,False,False,False,2,1,1,1,17,10
2,2017-01-08,0,0,12,256,3,0_12_256,2017,1,8,...,False,False,False,False,2,1,1,1,17,10
3,2017-01-08,0,0,12,354,3,0_12_354,2017,1,8,...,False,False,False,False,2,1,1,1,17,10
4,2017-01-08,0,0,13,149,2,0_13_149,2017,1,8,...,False,False,False,False,2,1,1,1,17,10


In [48]:
def categorical_to_numeric(df):
    cat_columns = df.select_dtypes(['category']).columns
    print(cat_columns)
    df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
    df[cat_columns] = df[cat_columns].astype(int)
    return df

In [49]:
print(df_sales.shape,df_submission_sample.shape)
df_auxiliar = pd.concat([df_sales,df_submission_sample])
df_auxiliar.replace([np.inf, -np.inf, np.nan],0,inplace=True)

df_auxiliar = string_to_categorical(df_auxiliar).copy()

dates = df_auxiliar['DATE'].unique()
dict_dates = {}
for idx,date in enumerate(dates):
    dict_dates[date]=idx
    
#df_auxiliar['date_block_num'] = df_auxiliar['DATE'].replace(dict_dates)
df_auxiliar['date_block_num'] = df_auxiliar['year']*12*4+df_auxiliar['month']*4+df_auxiliar['day']//7

df_auxiliar['date_block_num'] = df_auxiliar['date_block_num']-int(df_auxiliar['date_block_num'].min())+1

df_auxiliar = categorical_to_numeric(df_auxiliar)

df_auxiliar = columns_to_str(df_auxiliar,["S100","item_id","S101", "S102","S103","I100","I101","I102","I103","C100","C101"])

df_sales, df_submission_sample = df_auxiliar[:N_sales], df_auxiliar[N_sales:]
print(df_sales.shape,df_submission_sample.shape)

(2749369, 22) (69121, 23)
item_id object category
Index(['item_id', 'ID'], dtype='object')
(2749369, 24) (69121, 24)


In [50]:
features_names = set(df_sales.columns)-set(['QTT','DATE'])
print(features_names)

{'day', 'day_of_week', 'S100', 'item_id', 'day_of_year', 'C101', 'month', 'I101', 'I103', 'is_quarter_start', 'is_month_end', 'S102', 'ID', 'date_block_num', 'S101', 'C100', 'is_month_start', 'is_year_start', 'year', 'S103', 'I100', 'I102'}


In [51]:
df_auxiliar['date_block_num']

0          1
1          1
2          1
3          1
4          1
        ... 
69116    231
69117    233
69118    233
69119    230
69120    233
Name: date_block_num, Length: 2818490, dtype: int64

In [52]:
df_submission_sample['DATE'].min(),df_submission_sample['DATE'].max()

(Timestamp('2021-10-03 00:00:00'), Timestamp('2021-11-21 00:00:00'))

In [53]:
#df_sales2 = df_sales[df_sales['QTT']<=15].copy()

In [54]:
#df_submission_sample = df_submission_sample[df_submission_sample['S100']==0]

In [55]:
import warnings
import numpy as np
import pandas as pd
import copy
from pathlib import Path
import torch
import torch.nn as nn

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

import pytorch_forecasting
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [58]:
#'day_of_week', 'day', 'is_month_end', 'day_of_year',  'is_quarter_start', 'year', 'month', 'is_year_start', 'is_month_start'
for cat_col in ["S100", "item_id","S101", "S102","S103","I100","I101","I102","I103","C100","C101"]:
    df_sales[cat_col]             = df_sales[cat_col].astype(str).astype('category')
    df_submission_sample[cat_col] = df_submission_sample[cat_col].astype(str).astype('category')


In [59]:
df_sales.dtypes

DATE                datetime64[ns]
S100                      category
I100                      category
C100                      category
C101                      category
QTT                          int64
item_id                   category
year                         int64
month                        int64
day                          int64
day_of_week                  int64
day_of_year                  int64
is_year_start                 bool
is_quarter_start              bool
is_month_start                bool
is_month_end                  bool
I101                      category
I102                      category
I103                      category
S101                      category
S102                      category
S103                      category
ID                           int64
date_block_num               int64
dtype: object

In [62]:
#'day_of_week', 'day', 'is_month_end', 'day_of_year',  'is_quarter_start', 'year', 'month', 'is_year_start', 'is_month_start', 
# 'I103','S103', 'C101','I100' , 'C100', 'ID', 'I102','S102',, 'S101', 'S100', 'item_id', 'date_block_num', 'I101'
max_prediction_length = 6

max_encoder_length = 27

training_cutoff = df_sales['date_block_num'].max() - max_prediction_length

training = TimeSeriesDataSet(
    df_sales[lambda x: x['date_block_num'] <= training_cutoff],
    time_idx='date_block_num',
    target="QTT",
    group_ids=["S100", "item_id"],
    min_encoder_length=0,  
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["S101", "S102","S103","I100","I101","I102","I103","C100","C101"],
    static_reals=["S100", "item_id",'day_of_week', 'day', 'is_month_end', 'day_of_year',  'is_quarter_start', 'year', 'month', 'is_year_start', 'is_month_start'],
    time_varying_known_categoricals=[],  
    time_varying_known_reals=['date_block_num'],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=['date_block_num'],
    categorical_encoders={'S100': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True),
                          'item_id':pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True)},
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
    
)

KeyboardInterrupt: 

In [ ]:
validation = TimeSeriesDataSet.from_dataset(training, df_sales, predict=True, stop_randomization=True)
batch_size = 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader   = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

In [ ]:
# configure network and trainer
pl.seed_everything(42)
trainer = pl.Trainer(
    gpus=1,
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.03,
    hidden_size=16,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=1,
    dropout=0.1,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=8,  # set to <= hidden_size
    output_size=1,  # 7 quantiles by default
    loss=pytorch_forecasting.metrics.RMSE(),
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# find optimal learning rate
res = trainer.tuner.lr_find(
    tft,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
    max_lr=0.1,
    min_lr=1e-7,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

In [ ]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-7, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  
logger = TensorBoardLogger("lightning_logs") 

trainer = pl.Trainer(
    max_epochs=100,
    gpus=1,
    weights_summary="top",
    gradient_clip_val=0.1,
    limit_train_batches=30,  
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=5e-7,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=1, 
    loss=pytorch_forecasting.metrics.RMSE(),
    log_interval=10,  
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# fit network
trainer.fit(
    tft,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

In [ ]:
# load the best model according to the validation loss
# (given that we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
print(best_model_path)
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [ ]:
# calcualte root mean squared error on validation set
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
val_predictions = best_tft.predict(val_dataloader)

In [ ]:
criterion = nn.MSELoss()
torch.sqrt(criterion(actuals,val_predictions))

In [ ]:
for x, y in iter(val_dataloader):
    break

In [31]:
X_train[["S100","item_id","S101", "S102","S103","I100","I101","I102","I103","C100","C101"]]

NameError: name 'X_train' is not defined

In [ ]:
x['encoder_cat'][0]

In [ ]:
X_train_pred      = gbm.predict(X_train, num_iteration=gbm.best_iteration_)
X_validation_pred = gbm.predict(X_validation, num_iteration=gbm.best_iteration_)
X_test_pred       = gbm.predict(X_test, num_iteration=gbm.best_iteration_)

X_submission_pred = gbm.predict(X_submission, num_iteration=gbm.best_iteration_)

In [ ]:
from sklearn.metrics import mean_squared_error

X_train_pred = np.maximum(X_train_pred, 0)
X_validation_pred = np.maximum(X_validation_pred, 0)
X_test_pred = np.maximum(X_test_pred, 0)
X_submission_pred = np.maximum(X_submission_pred, 0)

df_train_rmse = mean_squared_error(y_train,X_train_pred, squared=False)
df_val_rmse   = mean_squared_error(y_validation,X_validation_pred, squared=False)
df_test_rmse  = mean_squared_error(y_test,X_test_pred, squared=False)

print('Final score mean_squared_error')
print('Score train:',df_train_rmse)
print('Score val  :',df_val_rmse)
print('Score test :',df_test_rmse)

In [ ]:
#X_test_pred[0] = 200

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(20,8))
plt.plot(X_test_pred,'b', alpha=0.7)
plt.plot(y_test.values,'r', alpha=0.4)
plt.legend(["prediction", "real"], loc ="upper right")

plt.show()

In [ ]:
df_submission_sample.reset_index(drop=True,inplace=True)
df_submission_sample['QTT'] = X_submission_pred
df_submission_sample.head()

In [ ]:
filename_submission= 'submission_NOFILL_good_'+str(np.round(df_test_rmse,4))+'_local.csv'
save = False
if save:
    print('saving..')
    print(filename_submission)
    df_submission_sample[['ID','QTT']].to_csv(os.path.join(PATH_RESULTS,'submissions','no_fill',filename_submission),index=None)

In [ ]:
result = pd.read_csv(os.path.join(PATH_RESULTS,'submissions','submission_2.1046_local.csv'))

In [ ]:
df_submission_sample['QTT_reference']= result['QTT'].copy()
df_submission_sample[['QTT_reference','QTT']][-20:]

In [ ]:
fig = plt.figure(figsize=(20,8))
plt.plot(df_submission_sample['QTT'],'b', alpha=0.7)
plt.plot(result['QTT'],'r', alpha=0.4)
plt.legend(["prediction", "real"], loc ="upper right")
plt.show()

In [ ]:
difference = mean_squared_error(df_submission_sample['QTT'],result['QTT'], squared=False)

print('Final score mean_squared_error')
print('Score train:',difference)